In [ ]:
import requests
import csv
from functools import reduce
import pandas as pd
import time
import numpy as np

In [ ]:
place_geoid = pd.read_csv("../us-place-over100pop.csv")
state_abbr = pd.read_csv("../stateabbr.csv")

In [ ]:
base = [place_geoid, state_abbr] # Merging state abbreviation
placecode = reduce(lambda left, right: pd.merge(left, right, on=['state'], how='left'), base)

# Creating city name
placecode['cityname'] = placecode['NAME'].str.split(',').str[0]
placecode["cityname"] = placecode["cityname"].str.rsplit(' ', n=1).str[0]

# Remove characters after '/' or '-'
placecode["cityname"] = placecode["cityname"].str.replace(r'[-/].*', '', regex=True)
placecode["cityname"] = placecode["cityname"].str.lower()

# Replace a specific city name
placecode["cityname"] = placecode["cityname"].replace('san buenaventura (ventura)', 'ventura')
placecode["cityname"] = placecode["cityname"].replace('urban honolulu', 'honolulu')

# Remove where cell value includes "CDP"
placecode = placecode[~placecode["cityname"].str.contains('cdp')]

# Subset the DataFrame to include only rows where the Address contains "St."
sub_placecode_st = placecode[placecode['cityname'].str.contains("st. ")]
sub_placecode_st['cityname'] = sub_placecode_st['cityname'].str.replace("st.", "saint")

 # Subset the DataFrame to include only rows where the cityname has an "apostrophes"
sub_placecode_ap = placecode[placecode['cityname'].str.contains("['']")]
sub_placecode_ap['cityname'] = sub_placecode_ap['cityname'].str.replace("'", "", regex=False)

# Subset the DataFrame to include only rows where the cityname ends with "city"
sub_placecode_city = placecode[placecode['cityname'].str.endswith(" city")]
sub_placecode_city['cityname'] = sub_placecode_city['cityname'].str.replace(" city", "", regex=False)

# Subset the DataFrame to include only rows where the cityname ends with "town"
sub_placecode_town = placecode[placecode['cityname'].str.endswith(" town")]
sub_placecode_town['cityname'] = sub_placecode_town['cityname'].str.replace(" town", "", regex=False)

# Subset the DataFrame to include only rows where the cityname ends with "village"
sub_placecode_village = placecode[placecode['cityname'].str.endswith(" village$")]
sub_placecode_village['cityname'] = sub_placecode_village['cityname'].str.replace(" village", "", regex=False)


In [ ]:
concatenated_df = pd.concat([placecode, sub_placecode_st, sub_placecode_ap, sub_placecode_city, sub_placecode_town, sub_placecode_village])

# Reset the index of the concatenated dataframe and drop the old index
concatenated_df.reset_index(drop=True, inplace=True)

# Make a copy of the concatenated dataframe and store it in the variable placecode
placecode = concatenated_df.copy()

# Remove characters after '/' or '-' in the 'cityname' column using regular expressions
placecode["cityname"] = placecode["cityname"].str.replace(r'[(].*', '', regex=True)

# Sort the dataframe by the values in the 'GEO_ID' column in ascending order
placecode = placecode.sort_values(by='GEO_ID')

# Reset the index of the sorted dataframe and drop the old index
placecode.reset_index(drop=True, inplace=True)

## 01 Generating Location Filters (Dictionaries Matching Fips/GEOID)

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

def create_city_filter(code):
    # Creating keys
    code["key1"] = code["cityname"] + " " + code["statename"]
    code["key2"] = code["cityname"] + " " + code["state_abbr"]
    code["key3"] = code["cityname"] + ", " + code["statename"]
    code["key4"] = code["cityname"] + ", " + code["state_abbr"]
    code["key5"] = code["cityname"] + "  " + code["statename"]
    code["key6"] = code["cityname"] + "  " + code["state_abbr"]
    code["key7"] = code["cityname"] + " , " + code["statename"]
    code["key8"] = code["cityname"] + " , " + code["state_abbr"]
    code["key9"] = code["cityname"]
    code["key10"] = code["cityname"]
    code["key11"] = code["cityname"]

    # Replace a "cityname" with NaN where population is less than 30000 only for key9 (dict09) - Find the exact match
    code.loc[code['Population'] <= 30000, 'key9'] = np.nan
    # Replace a "cityname" with NaN where population is less than 150,000 only for key10 (dict10) - Find substring
    code.loc[code['Population'] <= 150000, 'key10'] = np.nan
    # This is a column to detect the cities that share their names
    code.loc[code['Population'] <= 15000, 'key11'] = np.nan

    # Convert all keys to lowercase
    cols = ["key1", "key2", "key3", "key4", "key5", "key6", "key7", "key8", "key9", "key10", "key11"]
    for col in cols:
        code[col] = code[col].str.lower()

    '''
    creating three lists of the cities to exclude from searching tweets only with the city names
    '''

    # List 01. For cities with population greater than 15K, creating a dictionary like 'Fitchburg': ['Fitchburg city, Massachusetts', 'Fitchburg city, Wisconsin']
    dup_dict = defaultdict(list)
    for index, row in code.iterrows():
        dup_dict[row['key11']].append(row['NAME'])

    # List 02. Cities in other countries
    dup_list =  list(dup_dict.values())

    other_exc = ["Columbia city, Missouri", "Vancouver city, Washington", "Melbourne city, Florida",
                  "Amsterdam village, Ohio", "Amsterdam city, Missouri", "Hollywood city, Florida",
                  "Bristol city, Connecticut", "New Britain city, Connecticut", "Cicero town, Illinois",
                  "Edinburg city, Texas", "Carmel city, Indiana"]

    # List 03. CDPs
    cdp_lst = []
    for index, row in code.iterrows():
        # Check if the "NAME" column's substring contains "CDP"
        if 'CDP' in row['NAME']:
            # If it does, add the corresponding value from the "cityname" column to the list
            cdp_lst.append(row['cityname'])

    # Combine three lists and create the list of cities to exclude
    exc_list = dup_list + other_exc + cdp_lst

    # Replace values in key9 with NaN if NAME includes any of the items in the dup_list using numpy.where()
    code['key9'] = np.where(code['NAME'].isin(exc_list), float('NaN'), code['key9']) # find the exact match just using a city's name
    code['key10'] = np.where(code['NAME'].isin(exc_list), float('NaN'), code['key10']) # find the exact match just using a city's name

    # Creating dictionaries (filters to apply)
    dict01 = dict(zip(code.key1, code.GEO_ID))
    dict02 = dict(zip(code.key2, code.GEO_ID))
    dict03 = dict(zip(code.key3, code.GEO_ID))
    dict04 = dict(zip(code.key4, code.GEO_ID))
    dict05 = dict(zip(code.key5, code.GEO_ID))
    dict06 = dict(zip(code.key6, code.GEO_ID))
    dict07 = dict(zip(code.key7, code.GEO_ID))
    dict08 = dict(zip(code.key8, code.GEO_ID))

    # Creating the dictionary from GEO_ID and key9 (only with city names) but excluding rows where key9 is missing
    dict09 = {row['key9']: row['GEO_ID'] for index, row in code.dropna(subset=['key9']).iterrows()}
    dict10 = {row['key10']: row['GEO_ID'] for index, row in code.dropna(subset=['key10']).iterrows()}

    return dict01, dict02, dict03, dict04, dict05, dict06, dict07, dict08, dict09, dict10, code

# City Filters
dict01, dict02, dict03, dict04, dict05, dict06, dict07, dict08, dict09, dict10, code = create_city_filter(placecode)

In [ ]:
code.to_csv("../location_filter.csv", index=False)

In [ ]:
def create_state_filter(state_abbr):
    # Important: Descending order to fix the West Virginia issue
    state_abbr = state_abbr.sort_values(by=['state'], ascending=False, ignore_index=True)

    # Convert the 'Value' column to a string column
    state_abbr['state'] = state_abbr['state'].apply(lambda x: f'{x:02}')

    # Convert it to lowercase
    state_abbr["statename"] = state_abbr["statename"].str.lower()

    # Creating the dictionary from statename to state
    state_names = dict(zip(state_abbr.statename, state_abbr.state))

    return state_names

# State Filters
dc_othernames = {'washington dc': '11', 'washington, d.c.': '11', 'washington d.c.': '11'}
state_names = create_state_filter(state_abbr)

dc_othernames.update(state_names)
dict_state = dc_othernames

# The US filter
dict_usa = {"united states": "00", "usa": "00", "us": "00", "united states of america": "00"}

## 02. Location Fiter Functions (Both Substring & Exact Strings)

In [ ]:
# Define a function named 'match_substring_geoid'
def match_substring_geoid(df, filterdict):

    # Create a boolean mask that checks if the 'GEO_ID' column in the DataFrame 'df' has null values.
    mask = df['GEO_ID'].isnull()

    # Iterate through the DataFrame rows where 'GEO_ID' is null.
    for i, user_loc in df.loc[mask, 'user_loc'].items():
        # Try to find a value in the 'filterdict' dictionary that is a substring of 'user_loc'.
        # If found, assign the matching value to 'location_found'.
        location_found = next((filterdict[location] for location in filterdict.keys() if isinstance(location, str) and location in user_loc), None)

        # Check if a value (GEO_ID) was found in 'location_found'.
        if location_found:

            # Update the 'GEO_ID' column at the current row 'i' with the found value.
            df.at[i, 'GEO_ID'] = location_found

    # Calculate the count of missing values in the 'GEO_ID' column.
    missing_count = df["GEO_ID"].isna().sum()

    # Return the modified DataFrame 'df' and the count of missing values.
    return df, missing_count

In [ ]:
# Define a function named 'match_exact_string_geoidid'
def match_exact_string_geoidid(df, filterdict):

    # Create a boolean mask that checks if the 'GEO_ID' column in the DataFrame 'df' has null values.
    mask = df['GEO_ID'].isnull()

    # Iterate through the DataFrame rows where 'GEO_ID' is null.
    for i, user_loc in df.loc[mask, 'user_loc'].items():

        # Check if 'user_loc' exists as a key in the 'filterdict' dictionary.
        if user_loc in filterdict:

            # Update the 'GEO_ID' column at the current row 'i' with the value (state FIPS) from 'filterdict'.
            df.at[i, 'GEO_ID'] = filterdict[user_loc]

    # Calculate the count of missing values in the 'GEO_ID' column.
    missing_count = df['GEO_ID'].isna().sum()

    # Return the modified DataFrame 'df' and the count of missing values.
    return df, missing_count

In [ ]:
# Define a function named 'match_substring_geoid'
def match_substring_stateid(df, filterdict):

    # Create a boolean mask that checks if the 'state' column in the DataFrame 'df' has null values.
    mask = df['state'].isnull()

    # Iterate through the DataFrame rows where 'state' is null.
    for i, user_loc in df.loc[mask, 'user_loc'].items():
        # Try to find a value in the 'filterdict' dictionary that is a substring of 'user_loc'.
        # If found, assign the matching value to 'location_found'.
        location_found = next((filterdict[location] for location in filterdict.keys() if isinstance(location, str) and location in user_loc), None)

        # Check if a value (state) was found in 'location_found'.
        if location_found:

            # Update the 'state' column at the current row 'i' with the found value.
            df.at[i, 'state'] = location_found

    # Calculate the count of missing values in the 'state' column.
    missing_count = df["state"].isna().sum()

    # Return the modified DataFrame 'df' and the count of missing values.
    return df, missing_count

In [ ]:
# Define a function named 'match_exact_string_stateid'
def match_exact_string_stateid(df, filterdict):

    # Create a boolean mask that checks if the 'state' column in the DataFrame 'df' has null values.
    mask = df['state'].isnull()

    # Iterate through the DataFrame rows where 'state' is null.
    for i, user_loc in df.loc[mask, 'user_loc'].items():

        # Check if 'user_loc' exists as a key in the 'filterdict' dictionary.
        if user_loc in filterdict:

            # Update the 'state' column at the current row 'i' with the value (state FIPS) from 'filterdict'.
            df.at[i, 'state'] = filterdict[user_loc]

    # Calculate the count of missing values in the 'state' column.
    missing_count = df["state"].isna().sum()

    # Return the modified DataFrame 'df' and the count of missing values.
    return df, missing_count

## 03. Implementation: Import Data (Each Year)

In [ ]:
year = "2013"

file_path = f"../SolarSentiment_{year}_cleaned.csv"
df_tweet = pd.read_csv(file_path, lineterminator='\n')

df_tweet['GEO_ID'] = np.nan # city-level census GEO-ID
df_tweet["state"] = np.nan # state FIPS
df_tweet = df_tweet.astype({'GEO_ID': 'string', "state": 'string'})
df_tweet.user_loc = df_tweet.user_loc.astype(str)
df_tweet.user_loc = df_tweet.user_loc.str.lower()

missing_raw = len(df_tweet['GEO_ID'].isnull())
print(f"Number of Tweets without GEO_ID: {missing_raw}")

In [ ]:
def remove_tweets_international_includes(df, places_to_remove):
    # Create a boolean mask to identify rows where the "user_loc" column exactly matches any of the strings to be removed
    mask = df["user_loc"].isin(places_to_remove)

    # Remove rows with the specified strings from the DataFrame
    df = df[~mask]

    return df

def remove_tweets_countries_matched(df, countries_to_remove):
    # Combine the countries into a single regular expression pattern
    pattern = '|'.join(rf'\b{country}\b' for country in countries_to_remove)

    # Use the pattern to filter the DataFrame
    df = df[~df['user_loc'].str.contains(pattern, case=False)]

    return df

#remove_tweets_international_includes
places_to_remove = ["melbourne, australia", "cambridge, uk", "alberta, canada", "calgary, canada", "toronto, ontario", "midhurst, ontario", "buenos aires",
                     "budapest, hungary", "kitchener, ontario", "dover, kent", "bristol, england", "midhurst, ontario", "bristol, uk", "bristol uk", "ontario, canada"]
df_tweet = remove_tweets_international_includes(df_tweet, places_to_remove)

# remove_tweets_countries_matched
countries_to_remove = [" india", " australia", " new zealand", " united kingdom", " ireland", ", alberta", "alberta, ",
                       ", ontario", ", columbia", " ghana", " france", " spain", " russia", " lebanon", ", greece",
                       ", egypt", ", guatemala", ", england", ", canada", ", hungary", "yorkshire", "lancshire"]
df_tweet = remove_tweets_countries_matched(df_tweet, places_to_remove)

## 04. Process Data Using the Location Filters

### 01. Applying `dict01` (e.g., denver colorado) - find the exact match

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up01, missing_count01 = match_exact_string_geoidid(df_tweet, dict01)
newly_annot01 = missing_raw - missing_count01

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict01 in {year}: {newly_annot01}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 02 Applying `dict02` (e.g., denver co) - find the exact match

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up02, missing_count02 = match_exact_string_geoidid(df_tweet_up01, dict02)
newly_annot02 = missing_count01 - missing_count02

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict02 in {year}: {newly_annot02}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 03 Applying `dict 03` (e.g., denver, colorado) - find the exact match

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up03, missing_count03 = match_exact_string_geoidid(df_tweet_up02, dict03)
newly_annot03 = missing_count02 - missing_count03

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict03 in {year}: {newly_annot03}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 04 Applying `dict 04` (e.g., denver, co) - find the exact match

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up04, missing_count04 = match_exact_string_geoidid(df_tweet_up03, dict04)
newly_annot04 = missing_count03 - missing_count04

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict04 in {year}: {newly_annot04}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 05. Applying `dict05` (e.g., denver  colorado) - find the exact match - (double spaced)

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up05, missing_count05 = match_exact_string_geoidid(df_tweet_up04, dict05)
newly_annot05 = missing_count04 - missing_count05

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict05 in {year}: {newly_annot05}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 06 Applying `dict06` (e.g., denver  co) - find the exact match - (double spaced)

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up06, missing_count06 = match_exact_string_geoidid(df_tweet_up05, dict06)
newly_annot06 = missing_count05 - missing_count06

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict06 in {year}: {newly_annot06}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 07 Applying `dict 07` (e.g., denver , colorado) - find the exact match - (double spaced)

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up07, missing_count07 = match_exact_string_geoidid(df_tweet_up06, dict07)
newly_annot07 = missing_count06 - missing_count07

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict07 in {year}: {newly_annot07}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 08 Applying `dict 08` (e.g., denver , co) - find the exact match - (double spaced)

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up08, missing_count08 = match_exact_string_geoidid(df_tweet_up07, dict08)
newly_annot08 = missing_count07 - missing_count08

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict08 in {year}: {newly_annot08}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 09 Applying `dict 09` (e.g., denver) - find the exact match

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up09, missing_count09 = match_exact_string_geoidid(df_tweet_up08, dict09)
newly_annot09 = missing_count08 - missing_count09

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict09 in {year}: {newly_annot09}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 10 Applying `dict 10` (e.g., denver) - find substrings

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up10, missing_count10 = match_substring_geoid(df_tweet_up09, dict10)
newly_annot10 = missing_count09 - missing_count10

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated GEO_ID using dict10 (names of the cities), matching substrings in {year}: {newly_annot10}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 11 Applying `dict_state` (e.g., north carolina) - find the exact match

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up11, missing_count11 = match_exact_string_stateid(df_tweet_up10, dict_state)
newly_annot11 = missing_raw - missing_count11

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated State FIPS using dict_state in {year}: {newly_annot11}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 12 Applying `dict_state` (e.g., north carolina) - find substrings

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up12, missing_count12 = match_substring_stateid(df_tweet_up11, dict_state)
newly_annot12 = missing_count11 - missing_count12

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated State FIPS using dict_state, matching substrings in {year}: {newly_annot12}")
print(f"Total execution time: {process_duration:.2f} minutes")

### 13 Applying `dict_usa` (e.g., united states) - find the exact match

In [ ]:
# Start timing
start_time = time.time()
df_tweet_up13, missing_count13 = match_exact_string_stateid(df_tweet_up12, dict_usa)
newly_annot13 = missing_count12 - missing_count13

# Calculate the total execution time for the whole cell
process_duration = (time.time() - start_time) / 60

print(f"Newly annotated State FIPS using dict_usa in {year}: {newly_annot13}")
print(f"Total execution time: {process_duration:.2f} minutes")

## 05. Save

In [ ]:
import re

def clean_subset_dataframe(df):
    # Remove rows where either 'GEO_ID' or 'state' column has a value
    df = df[df['GEO_ID'].notna() | df['state'].notna()]

    # Extract State Fips column (two digits following "US" in GEO_ID)
    df.loc[:, "STATE_FIPS"] = df["GEO_ID"].str[9:11]

    # Replace STATE_FIPS with state ID where it is NaN
    df.loc[df["STATE_FIPS"].isna(), "STATE_FIPS"] = df['state']

    # Drop the 'state' column
    df = df.drop("state", axis=1)

    # Replace all words starting with "@" in the 'text' column with "[NAME]"
    df['text'] = df['text'].str.replace(r'@\w+', '[NAME]', regex=True)

    # Remove URLs from the 'text' column
    df['text'] = df['text'].str.replace(r'\s*http://\S+(\s+|$)', '', regex=True).str.strip()

    # Replace all phone numbers in the 'text' column with "[PHONE]"
    df['text'] = df['text'].str.replace(r'(\d{3})[-.\s]?(\d{3})[-.\s]?(\d{4})', '[PHONE]', regex=True)

    # Get tweet text length
    df["text_length"] = df.text.str.len()

    # Remove meaningless tweets (text length > 5)
    df = df[df["text_length"] > 5]

    # Drop the 'text_length' column
    df = df.drop("text_length", axis=1)

    df_pred = df[["tweet_id", "text"]]

    df_left = df[['tweet_id', 'created_at', 'user_id', 'user_loc', 'like_count',
       'retweet_count', 'referenced_type', 'referenced_id',
       'referenced_tweet', 'GEO_ID', 'STATE_FIPS']]

    return df, df_pred, df_left

df, df_pred, df_left = clean_subset_dataframe(df_tweet_up13)

In [ ]:
df_left.to_csv("../" + year + ".csv", index=False)

df_pred.to_csv("../" + year + ".csv", index=False)